In [1]:
%%HTML
<style>
   div#notebook-container    { width: 95%; }
   div#menubar-container     { width: 65%; }
   div#maintoolbar-container { width: 99%; }
</style>

In [2]:
import sys
sys.path.append("../..") # Adds higher directory to python modules path.
from utilities import aggregate_feature_calculators
from utilities import aggregate_feature_calculators_setting as aggcal
from utilities.parallel import Parallel
from utilities.dfdb import DFDB

from utilities.process.pqueue import *
from utilities.process.pnode import *
from utilities.process.putilities import *

Using TensorFlow backend.


In [3]:
import numpy as np
import pandas as pd
import os
import time
import datetime
import json
import copy
import gc
import warnings
from tqdm import tqdm_notebook, tqdm

import optuna

import lightgbm as lgb
import xgboost as xgb

from catboost import CatBoostRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold, KFold, RepeatedKFold,TimeSeriesSplit, GroupKFold
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn import linear_model
from sklearn.preprocessing import StandardScaler
from sklearn.svm import NuSVR, SVR
from sklearn.metrics import mean_absolute_error
from sklearn import preprocessing

import eli5
from eli5.sklearn import PermutationImportance

import networkx as nx

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [4]:
csv_file_folder =  '../../data/input'
os.listdir(csv_file_folder)

['sample_submission.csv',
 'magnetic_shielding_tensors.csv',
 'potential_energy.csv',
 'scalar_coupling_contributions.csv',
 'dipole_moments.csv',
 'mulliken_charges.csv',
 'train.csv',
 'test.csv',
 'structures.csv',
 'structures']

In [5]:
file_folder =  '../../data/feature'
[f for f in os.listdir(file_folder) if (f.endswith('.pkl')) and (not f.startswith('.'))]

[]

In [ ]:
y=fc, type=0
227
{'columns': ['atom_index_1_cycle_size_mean', 'dist_H_0_y', 'dist_H_1_x', 'tertiary_atom_1', 'adC2', 'tertiary_distance_1', 'dist_C_2_y', 'molecule_atom_index_1_dist_min_diff', 'dist_C_1_y', 'atom_index_farthest_0', 'yukawa_H.y', 'inv_dist1R', 'dist_C_0_x', 'dist_H_3_x', 'tertiary_angle_3', 'dist_to_type_std', 'dist_C_3_y', 'eem_1', 'inv_distPR', 'dist_C_1_x', 'linkM0', 'dist_H_2_y', 'dist_H_0_x', 'adC1', 'tertiary_angle_0', 'adN1', 'dist_O_1_y', 'dist_C_2_x', 'eem_0', 'dist_C_4_x', 'adC3', 'tertiary_distance_3', 'dist_O_0_x', 'tertiary_angle_2', 'atom_1_bond_lengths_std', 'molecule_atom_index_0_dist_max_div', 'inv_dist0R', 'distance_farthest_0', 'yukawa_N.x', 'dist_H_3_y', 'molecule_type_dist_mean_diff', 'dist_N_0_x', 'max_molecule_atom_1_dist_xyz', 'dist_C_0_y', 'dist_H_2_x', 'dist_to_type_0_mean', 'tertiary_distance_5', 'atom_1_n_bonds', 'dist_C_3_x', 'tertiary_distance_0', 'dist_to_type_mean', 'distC1', 'dist_O_0_y', 'dist_C_4_y', 'tertiary_distance_2', 'cos_f0', 'atom_1_bond_lengths_max', 'dist_N_0_y', 'dist_xyz', 'dist_H_1_y', 'dist_O_1_x', 'tertiary_distance_4', 'atom_1_bond_lengths_mean', 'yukawa_H.x', 'tertiary_atom_0', 'tertiary_angle_4', 'inv_dist1E', 'tertiary_angle_1'], 'cv': {'cls': 'KFold', 'init': {'n_splits': 5, 'shuffle': True, 'random_state': 42}}, 'scaler': {'cls': 'StandardScaler', 'init': {}, 'fit': {}}, 'model': {'cls': 'lgb.LGBMRegressor', 'init': {'learning_rate': 0.2833769330240482, 'feature_fraction': 0.8818248470204605, 'bagging_fraction': 0.8205197060908092, 'min_data_in_leaf': 202, 'lambda_l1': 0.017039063121824582, 'lambda_l2': 0.8318702431636841, 'max_bin': 100, 'num_leaves': 255, 'random_state': 3895, 'n_jobs': 16}, 'fit': {}}, 'metric': 'mean_absolute_error'}
y=fc, type=1
561
{'columns': ['atom_index_1_cycle_size_mean', 'dist_H_0_y', 'dist_H_1_x', 'linkM1', 'adC2', 'tertiary_distance_1', 'tertiary_distance_6', 'dist_C_2_y', 'distance_center0', 'dist_C_1_y', 'yukawa_H.y', 'inv_dist1R', 'dist_C_0_x', 'yukawa_C.x', 'inv_dist1', 'cos_center0', 'tertiary_angle_3', 'cos_center1', 'dist_C_3_y', 'vander_O.y', 'eem_1', 'inv_distPR', 'dist_C_1_x', 'inv_distP', 'vander_N.y', 'dist_H_2_y', 'dist_H_0_x', 'tertiary_angle_0', 'vander_N.x', 'dist_C_2_x', 'eem_0', 'coulomb_H.x', 'min_molecule_atom_1_dist_xyz', 'yukawa_N.y', 'molecule_atom_index_0_dist_mean_div', 'dist_C_4_x', 'adC3', 'tertiary_distance_3', 'linkN', 'dist_O_0_x', 'max_molecule_atom_0_dist_xyz', 'distance_center1', 'tertiary_angle_2', 'atom_1_bond_lengths_std', 'molecule_atom_index_0_dist_max_div', 'inv_dist0R', 'link0', 'yukawa_N.x', 'dist_H_3_y', 'dist_N_0_x', 'dist_C_0_y', 'molecule_atom_1_dist_min_diff', 'dist_to_type_0_mean', 'inv_distPE', 'vander_H.x', 'tertiary_distance_5', 'dist_N_1_y', 'dist_N_1_x', 'dist_C_3_x', 'tertiary_distance_0', 'dist_O_0_y', 'distN0', 'dist_C_4_y', 'cos_f1', 'tertiary_distance_2', 'vander_O.x', 'cos_f0', 'atom_1_bond_lengths_max', 'dist_N_0_y', 'dist_H_1_y', 'tertiary_distance_4', 'atom_1_bond_lengths_mean', 'tertiary_atom_0', 'distN1', 'tertiary_angle_1', 'adC4', 'bond_atom'], 'cv': {'cls': 'KFold', 'init': {'n_splits': 5, 'shuffle': True, 'random_state': 42}}, 'scaler': {'cls': 'StandardScaler', 'init': {}, 'fit': {}}, 'model': {'cls': 'lgb.LGBMRegressor', 'init': {'learning_rate': 0.2833769330240482, 'feature_fraction': 0.8818248470204605, 'bagging_fraction': 0.8205197060908092, 'min_data_in_leaf': 202, 'lambda_l1': 0.017039063121824582, 'lambda_l2': 0.8318702431636841, 'max_bin': 100, 'num_leaves': 255, 'random_state': 3895, 'n_jobs': 16}, 'fit': {}}, 'metric': 'mean_absolute_error'}
y=fc, type=2
788
{'columns': ['atom_index_1_cycle_size_mean', 'atom_0_bond_lengths_max', 'dist_H_0_y', 'tertiary_atom_1', 'adC2', 'tertiary_atom_2', 'tertiary_distance_6', 'tertiary_distance_1', 'dist_C_2_y', 'molecule_atom_index_1_dist_min_diff', 'dist_C_1_y', 'dist_C_0_x', 'inv_dist1R', 'atom_1_bond_lengths_min', 'cos_c0_c1', 'tertiary_angle_3', 'inv_dist1', 'molecule_atom_index_0_dist_max_diff', 'dist_C_3_y', 'vander_O.y', 'eem_1', 'inv_distPR', 'dist_C_1_x', 'dist_H_0_x', 'adC1', 'tertiary_angle_0', 'adN1', 'dist_C_2_x', 'molecule_atom_index_0_dist_mean_div', 'molecule_atom_index_0_dist_min_diff', 'adC3', 'tertiary_distance_3', 'dist_O_0_x', 'max_molecule_atom_0_dist_xyz', 'tertiary_angle_2', 'atom_1_bond_lengths_std', 'inv_dist0R', 'distance_farthest_0', 'dist_N_0_x', 'dist_C_0_y', 'atom_index_1_n_cycle', 'molecule_atom_1_dist_min_diff', 'dist_to_type_0_mean', 'cos_c0', 'distC0', 'tertiary_atom_3', 'bond_atom', 'molecule_atom_index_1_dist_min_div', 'atom_1_n_bonds', 'vander_C.y', 'dist_C_3_x', 'tertiary_atom_4', 'dist_O_0_y', 'molecule_atom_index_0_dist_min_div', 'tertiary_distance_2', 'cos_f0', 'atom_1_bond_lengths_max', 'dist_N_0_y', 'dist_xyz', 'dist_H_1_y', 'dist_O_1_x', 'atom_1_bond_lengths_mean', 'min_molecule_atom_0_dist_xyz', 'molecule_type_dist_max', 'tertiary_atom_0', 'tertiary_angle_4', 'tertiary_angle_1', 'cos_c1'], 'cv': {'cls': 'KFold', 'init': {'n_splits': 5, 'shuffle': True, 'random_state': 42}}, 'scaler': {'cls': 'StandardScaler', 'init': {}, 'fit': {}}, 'model': {'cls': 'lgb.LGBMRegressor', 'init': {'learning_rate': 0.2833769330240482, 'feature_fraction': 0.8818248470204605, 'bagging_fraction': 0.8205197060908092, 'min_data_in_leaf': 202, 'lambda_l1': 0.017039063121824582, 'lambda_l2': 0.8318702431636841, 'max_bin': 100, 'num_leaves': 255, 'random_state': 3895, 'n_jobs': 16}, 'fit': {}}, 'metric': 'mean_absolute_error'}
y=fc, type=3
294
{'columns': ['yukawa_H.y', 'dist_H_0_y', 'dist_H_3_x', 'tertiary_distance_2', 'dist_H_1_y', 'tertiary_angle_0', 'vander_H.y', 'adC3', 'dist_H_1_x', 'tertiary_distance_3', 'tertiary_atom_2', 'dist_to_type_0_mean', 'molecule_atom_index_0_dist_min_div', 'tertiary_angle_3', 'dist_H_2_x', 'dist_to_type_std', 'dist_C_0_y', 'tertiary_distance_1', 'adN1', 'molecule_atom_index_0_dist_mean_diff', 'inv_distPE', 'adC1', 'molecule_type_dist_max', 'cos_f0', 'eem_1', 'tertiary_atom_1', 'dist_C_3_y', 'molecule_atom_index_0_dist_min_diff', 'dist_N_0_y', 'cos_c1', 'molecule_dist_min', 'vander_H.x', 'mean_molecule_atom_1_dist_xyz', 'dist_H_2_y', 'dist_to_type_1_mean', 'dist_O_0_y', 'molecule_atom_index_0_dist_max_div', 'dist_C_3_x', 'cos_c0', 'mean_molecule_atom_0_dist_xyz', 'cos_f0_f1', 'cos_c0_c1', 'inv_distPR', 'dist_N_0_x', 'dist_C_1_x', 'dist_H_4_x', 'dist_O_0_x', 'dist_H_4_y', 'adC2', 'cos_f1', 'dist_C_0_x', 'min_molecule_atom_1_dist_xyz', 'dist_H_3_y', 'dist_C_1_y', 'max_distance_y', 'dist_C_2_y', 'dist_to_type_mean', 'yukawa_H.x', 'dist_xyz', 'dist_H_0_x', 'distance_c1', 'inv_distP', 'link0', 'tertiary_distance_4', 'tertiary_angle_1', 'distance_farthest_0', 'dist_C_2_x'], 'cv': {'cls': 'KFold', 'init': {'n_splits': 5, 'shuffle': True, 'random_state': 42}}, 'scaler': {'cls': 'StandardScaler', 'init': {}, 'fit': {}}, 'model': {'cls': 'lgb.LGBMRegressor', 'init': {'learning_rate': 0.2833769330240482, 'feature_fraction': 0.8818248470204605, 'bagging_fraction': 0.8205197060908092, 'min_data_in_leaf': 202, 'lambda_l1': 0.017039063121824582, 'lambda_l2': 0.8318702431636841, 'max_bin': 100, 'num_leaves': 255, 'random_state': 3895, 'n_jobs': 16}, 'fit': {}}, 'metric': 'mean_absolute_error'}
y=fc, type=4
468
{'columns': ['yukawa_H.y', 'dist_H_0_y', 'dist_H_3_x', 'tertiary_distance_2', 'dist_H_1_y', 'tertiary_angle_0', 'vander_H.y', 'adC3', 'dist_H_1_x', 'tertiary_distance_3', 'tertiary_atom_2', 'dist_to_type_0_mean', 'molecule_atom_index_0_dist_min_div', 'tertiary_angle_3', 'dist_H_2_x', 'dist_to_type_std', 'dist_C_0_y', 'tertiary_distance_1', 'adN1', 'molecule_atom_index_0_dist_mean_diff', 'inv_distPE', 'adC1', 'molecule_type_dist_max', 'cos_f0', 'eem_1', 'tertiary_atom_1', 'dist_C_3_y', 'molecule_atom_index_0_dist_min_diff', 'dist_N_0_y', 'cos_c1', 'molecule_dist_min', 'vander_H.x', 'mean_molecule_atom_1_dist_xyz', 'dist_H_2_y', 'dist_to_type_1_mean', 'dist_O_0_y', 'molecule_atom_index_0_dist_max_div', 'dist_C_3_x', 'cos_c0', 'mean_molecule_atom_0_dist_xyz', 'cos_f0_f1', 'cos_c0_c1', 'inv_distPR', 'dist_N_0_x', 'dist_C_1_x', 'dist_H_4_x', 'dist_O_0_x', 'dist_H_4_y', 'adC2', 'cos_f1', 'dist_C_0_x', 'min_molecule_atom_1_dist_xyz', 'dist_H_3_y', 'dist_C_1_y', 'max_distance_y', 'dist_C_2_y', 'dist_to_type_mean', 'yukawa_H.x', 'dist_xyz', 'dist_H_0_x', 'distance_c1', 'inv_distP', 'link0', 'tertiary_distance_4', 'tertiary_angle_1', 'distance_farthest_0', 'dist_C_2_x'], 'cv': {'cls': 'KFold', 'init': {'n_splits': 5, 'shuffle': True, 'random_state': 42}}, 'scaler': {'cls': 'StandardScaler', 'init': {}, 'fit': {}}, 'model': {'cls': 'lgb.LGBMRegressor', 'init': {'learning_rate': 0.2833769330240482, 'feature_fraction': 0.8818248470204605, 'bagging_fraction': 0.8205197060908092, 'min_data_in_leaf': 202, 'lambda_l1': 0.017039063121824582, 'lambda_l2': 0.8318702431636841, 'max_bin': 100, 'num_leaves': 255, 'random_state': 3895, 'n_jobs': 16}, 'fit': {}}, 'metric': 'mean_absolute_error'}
y=fc, type=6

In [6]:
mytrial = []

In [ ]:
# ''scalar_coupling_constant, 'fc', 'sd','pso','dso'
for y in ['fc']:
    for t in np.arange(5,8,1):
        
        print(f'y={y}, type={t}')
        df_train=pd.read_pickle(f'{file_folder}/df_train.gzde', compression='gzip')
#         df_test=pd.read_pickle(f'{file_folder}/df_test.gzde', compression='gzip')
        
        df_train['y'] = df_train[y]
        df_train = df_train[df_train['type']==t]
        n_samples = 500000 if df_train.shape[0] > 500000 else df_train.shape[0]
        df_train = df_train.sample(n_samples).reset_index(drop=True)
#         df_test = df_test[df_test['type']==t].reset_index(drop=True)
        df_test = pd.DataFrame()
        

        param = {
            'columns': df_train.columns.drop(['index', 'y','group', 'scalar_coupling_constant', 'fc', 'sd','pso','dso']).tolist(),
            'cv': {
                'cls': 'KFold',
                'init':{
                    'n_splits': 5,
                    'shuffle': True,
                    'random_state': 42,
                },
            },
            'scaler': {'cls': 'StandardScaler', 'init': {}, 'fit': {}},
            'model': {
                'cls': 'lgb.LGBMRegressor',
                'init': {
                    'learning_rate': 0.2833769330240482,
                    'feature_fraction': 0.8818248470204605,
                    'bagging_fraction': 0.8205197060908092,
                    'min_data_in_leaf': 202,
                    'lambda_l1': 0.017039063121824582,
                    'lambda_l2': 0.8318702431636841,
                    'max_bin': 100,
                    'num_leaves': 255,
                    'random_state': 3895,
                    'n_jobs': 16
                },
                'fit': {}
            },
            'metric': 'mean_absolute_error'
        }



        process_queue = PQueue(df_train, df_test, param, mytrial)
        sort_features = SortFeatureSelectTopNProcess(**{'top_n':200})
        select_topn = RFESelectTopNProcess(**{'n_features_remain':20, 'n_features_to_remove':10})
        remove_useless = RFERemoveUselessFeaturesProcess(**{})
        process_queue.insert_node(sort_features)
        process_queue.insert_node(select_topn)
        process_queue.insert_node(remove_useless)

        try:
            result = process_queue.run()
        except Exception as e:
            print(e.__str__())
        print(len(process_queue.trial))
        print(process_queue.param)
    

y=fc, type=5


In [11]:
df_train=pd.read_pickle(f'{file_folder}/df_train.gzde', compression='gzip')
df_test=pd.read_pickle(f'{file_folder}/df_test.gzde', compression='gzip')
df_train['y'] = df_train['fc']

In [12]:
columns_list = []

In [13]:
mytrial=[]
df_his, df_feature_importances, df_valid_pred, df_test_pred = pd.DataFrame(), pd.DataFrame(),pd.DataFrame(),pd.DataFrame()
for t in  df_train.type.unique().tolist():
    df_his_i, df_feature_importances_i, df_valid_pred_i, df_test_pred_i =  sk_process(df_train[df_train['type']==t].reset_index(drop=True), columns_list[t], f'modeling for {t}', df_test=df_test[df_test['type']==t].reset_index(drop=True), trial=mytrial, is_output_feature_importance=False, trial_level=1)
    df_his = pd.concat([df_his, df_his_i], axis=0)
    df_feature_importances = pd.concat([df_feature_importances, df_feature_importances_i], axis=0)
    df_valid_pred = pd.concat([df_valid_pred, df_valid_pred_i], axis=0)
    df_test_pred = pd.concat([df_test_pred, df_test_pred_i], axis=0)

df_valid_pred = df_valid_pred.sort_values(by=['index']).reset_index(drop=True)
df_test_pred = df_test_pred.sort_values(by=['index']).reset_index(drop=True)

In [18]:
df_trial = pd.DataFrame(mytrial)
df_trial[['datetime', 'message', 'nfeatures', 'train_metric_mean', 'val_metric_mean', 'trn_val_metric_diff', 'message']]

,datetime,message,nfeatures,train_metric_mean,val_metric_mean,trn_val_metric_diff,message
0,2019-07-16 11:18:23.406847,modeling for 0,78,0.820160,0.933006,0.112845,modeling for 0
1,2019-07-16 11:18:40.952761,modeling for 3,59,0.161547,0.200190,0.038643,modeling for 3
2,2019-07-16 11:18:43.612953,modeling for 1,58,0.286269,0.433112,0.146842,modeling for 1
3,2019-07-16 11:18:50.776218,modeling for 4,59,0.154898,0.237684,0.082786,modeling for 4
4,2019-07-16 11:19:50.783660,modeling for 2,68,0.360945,0.391322,0.030378,modeling for 2
5,2019-07-16 11:20:19.215675,modeling for 6,59,0.196629,0.225852,0.029223,modeling for 6
6,2019-07-16 11:21:38.304023,modeling for 5,59,0.427043,0.453386,0.026343,modeling for 5
7,2019-07-16 11:21:46.386701,modeling for 7,49,0.102380,0.141882,0.039502,modeling for 7


In [16]:
print(mean_absolute_error(df_valid_pred.sort_values(by=['index']).reset_index(drop=True).predict.values, df_train.reset_index(drop=True).y.values))
for t in df_train.type.unique().tolist():
    index = df_train[df_train['type']==t]['index'].values
    print(t, mean_absolute_error(df_valid_pred[df_valid_pred['index'].isin(index)].sort_values(by=['index']).reset_index(drop=True).predict.values, df_train[df_train['index'].isin(index)].reset_index(drop=True).y.values))

0.44499493261917566
0 0.9330056960257216
3 0.2001904951609826
1 0.4331118692460489
4 0.23768410461908276
2 0.39132241115217037
6 0.22585247088385893
5 0.4533857586621615
7 0.14188187674382496


In [22]:
df_test_pred.tail()

,index,0,1,2,3,4
2505537,7163684,0.210720,0.871862,0.746697,1.206203,1.007102
2505538,7163685,3.809454,3.867529,2.588954,2.918856,3.701579
2505539,7163686,3.658385,3.428395,3.517420,1.978553,3.407225
2505540,7163687,4.435589,4.727233,5.057265,4.668085,4.264797
2505541,7163688,118.513458,118.589493,116.857854,117.165564,112.934275


In [23]:
idx=0.44
# df_test_pred = df_trial.loc[idx]['df_test_pred']
df_submit = pd.DataFrame()
df_submit['scalar_coupling_constant'] = np.mean(df_test_pred.drop(columns=['index']).values, axis=1)
df_submit['id'] = df_test_pred['index']
df_submit.to_csv('../../data/submission/submission_lgbm_{}.csv'.format(idx), index=False)